In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from epiweeks import Week
from datetime import date, datetime
from os.path import dirname, join, isfile, isdir

from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf, normalize_igbe, normalize_labels
from bulletin.commom.utils import isvaliddate

from bulletin.commom.utils import Timer, auto_fit_columns
output = join("output")

In [2]:
internamento = pd.read_excel(join('input','internamento.xls'))
internamento.columns = [ normalize_labels(x) for x in internamento.columns]
internamento['leito_tipo'] = internamento['leito_tipo'].apply(normalize_text)
internamento.loc[internamento['leito_tipo']=='CORONAVIRUS-UTI ADULTO (MS)', 'leito_tipo'] = 'CORONAVIRUS-UTI ADULTO'
internamento['motivo_alta'] = internamento['motivo_alta'].apply(normalize_text)
# internamento['ano'] = internamento.apply(lambda row: Week.fromdate(row['data_alta']).year, axis=1)
# internamento['mes'] = internamento.apply(lambda row: Week.fromdate(row['data_alta']).startdate().month, axis=1)
# internamento['se'] = internamento.apply(lambda row: Week.fromdate(row['data_alta']).week, axis=1)
internamento

,data_alta,leito_tipo,motivo_alta,tempo_internamento_(em_dias),idade
0,2020-04-01 22:00:00.000,CORONAVIRUS-CLINICA GERAL,ALTA MEDICA,0,68
1,2020-04-02 16:00:00.000,CORONAVIRUS-CLINICA GERAL,ALTA MEDICA,7,46
2,2020-04-04 15:00:00.000,CORONAVIRUS-CLINICA GERAL,ALTA MEDICA,10,77
3,2020-04-05 17:00:00.000,CORONAVIRUS-CLINICA GERAL,ALTA MEDICA,4,32
4,2020-04-07 09:00:00.000,CORONAVIRUS-UTI ADULTO,ALTA MEDICA,7,62
...,...,...,...,...,...
1958,2021-03-03 00:56:05.375,CORONAVIRUS-CLINICA GERAL,ALTA POR TRANSFERENCIA PARA OUTRO HOSPITAL,3,30
1959,2021-03-03 00:56:19.107,CORONAVIRUS-CLINICA GERAL,ALTA POR TRANSFERENCIA PARA OUTRO HOSPITAL,3,67
1960,2021-03-03 01:00:00.000,CORONAVIRUS-CLINICA GERAL,ALTA MEDICA,4,42
1961,2021-03-03 01:13:40.443,CORONAVIRUS-CLINICA GERAL,ALTA POR TRANSFERENCIA PARA OUTRO HOSPITAL,2,52


In [3]:
media_internamento = internamento.groupby('leito_tipo')[['idade','tempo_internamento_(em_dias)']].mean()
media_internamento

,idade,tempo_internamento_(em_dias)
leito_tipo,,
CORONAVIRUS-CLINICA GERAL,58.218421,7.586842
CORONAVIRUS-UTI ADULTO,66.042889,10.855530


In [4]:
alta_internamento = internamento.groupby('motivo_alta')[['idade']].count().rename(columns={'idade':'qtde'})
aux_df = pd.DataFrame(columns=['qtde'], index=['TOTAL'])
aux_df.loc['TOTAL'] = alta_internamento.sum()
alta_internamento = alta_internamento.append(aux_df)
alta_internamento['percent'] = alta_internamento.apply(lambda row: row['qtde'] / alta_internamento.loc['TOTAL','qtde'], axis=1)
alta_internamento

,qtde,percent
ALTA ADMINISTRATIVA,10,0.005094
ALTA MEDICA,1468,0.747835
ALTA POR OBITO,379,0.193072
ALTA POR PEDIDO DO PACIENTE OU FAMILIAR,4,0.002038
ALTA POR TRANSFERENCIA PARA OUTRO HOSPITAL,97,0.049414
EVASAO DO PACIENTE,5,0.002547
TOTAL,1963,1.000000


In [5]:
writer = pd.ExcelWriter(join(output,"internamentos.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

media_internamento.to_excel(writer,sheet_name='media_internamento')
alta_internamento.to_excel(writer,sheet_name='alta_internamento')

worksheet = writer.sheets[f"media_internamento"]
auto_fit_columns(worksheet,media_internamento)

worksheet = writer.sheets[f"alta_internamento"]
auto_fit_columns(worksheet,alta_internamento)

writer.save()
writer.close()